In [42]:
import sys
import os
from torch.utils.data import DataLoader
sys.path.append("..")
from tqdm import tqdm
import shutil
import torch
import numpy as np

from utils import train_test_split, create_metadata_df
from dataset_xy import CARLADatasetXY
from dataset import CARLADataset
from data_preprocessing import preprocessing, transform_lidar_bev

from PIL import Image

In [43]:
path_data = os.path.join("..", "..", "data", "data")
# path_data = "../../data/data/cycl_dataset_23_11/Routes_non-straight-junction-Scen4_Town07_junction_Seed2000/non-straight-junction-Scen4_Town07_junction_route74_11_24_00_29_14"
# path_data = os.path.join("..", "..", "data", "data", "rr_dataset_23_11")
#path_data = os.path.join("..", "..", "data", "data_prep_npy")

config_xy = {"used_inputs": ["rgb", "lidar_prep", "measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake", "command"],
        "y": ["brake", "steer", "throttle"],
        "seq_len": 1
        }

# Create df_meta 
df_meta_data = create_metadata_df(path_data, config_xy["used_inputs"])

# Create Dataset & DataLoader
dataset_rgb = CARLADatasetXY(root_dir=path_data, df_meta_data=df_meta_data, config=config_xy)

In [16]:
df_meta_data.isna().sum()

dir                0
rgb                0
rgb_prep        6197
lidar_prep         0
measurements       0
dtype: int64

In [11]:
def check_if_in_every_folder_is_same_number_of_input_folders(path_data, num_expected=5):
    sus_paths = []
    for (root, dirs, files) in os.walk(path_data, topdown=True):
    
        # Current folder contains the files
        if not dirs:
            folder, input = os.path.split(root)
            potential_inputs = os.listdir(folder)
            potential_inputs = [input for input in potential_inputs if os.path.isdir(os.path.join(folder, input))]
            if len(potential_inputs) != num_expected:
                sus_paths.append(folder)
    return sus_paths

In [18]:
sus_paths = check_if_in_every_folder_is_same_number_of_input_folders(path_data, num_expected=5)
len(sus_paths)

0

In [21]:
df_meta_data[df_meta_data["rgb_prep"].isna()]["dir"].unique

<bound method Series.unique of 1678      ../../data/data/cycl_dataset_23_11/Routes_non-...
1679      ../../data/data/cycl_dataset_23_11/Routes_non-...
1680      ../../data/data/cycl_dataset_23_11/Routes_non-...
1681      ../../data/data/cycl_dataset_23_11/Routes_non-...
1682      ../../data/data/cycl_dataset_23_11/Routes_non-...
                                ...                        
256120    ../../data/data/dirt_dataset_23_11/Routes_Scen...
256121    ../../data/data/dirt_dataset_23_11/Routes_Scen...
256122    ../../data/data/dirt_dataset_23_11/Routes_Scen...
256123    ../../data/data/dirt_dataset_23_11/Routes_Scen...
256124    ../../data/data/dirt_dataset_23_11/Routes_Scen...
Name: dir, Length: 6197, dtype: object>

In [ ]:
df_meta_data[df_meta_data["rgb_prep"].isna()]["dir"].unique()

In [38]:
len(os.listdir('../../data/data/cycl_dataset_23_11/Routes_non-straight-junction-Scen4_Town07_junction_Seed2000/non-straight-junction-Scen4_Town07_junction_route89_11_24_01_07_09/rgb'))

206

In [40]:
files_prob = sorted(os.listdir('../../data/data/cycl_dataset_23_11/Routes_non-straight-junction-Scen4_Town07_junction_Seed2000/non-straight-junction-Scen4_Town07_junction_route89_11_24_01_07_09/rgb_prep'))

In [41]:
# Problem found: for some files .npz and .npy are saved --> this makes the difference between the number of files
# Solution: delete entire rgb_prep (I don't need it anyways at the moment)
[file for file in files_prob if file.endswith("npz")]

['0110.npz']